In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from helpers.helper_functions import *

import spacy

# Handling text 2 exercise
[Handling text exercisses ADApted drom ADA 2018 final exam]

The Sheldon Cooper we all know and love (OK, some of us might not know him, and some might not love him) from the TV series "The Big Bang Theory" has gotten into an argument with Leonard from the same TV show. Sheldon insists that he knows the show better than anyone, and keeps making various claims about the show, which neither of them know how to prove or disprove. The two of them have reached out to you ladies and gentlemen, as data scientists, to help them. You will be given the full script of the series, with information on the episode, the scene, the person saying each dialogue line, and the dialogue lines themselves.

Leonard has challenged several of Sheldon's claims about the show, and throughout this exam you will see some of those and you will get to prove or disprove them, but remember: sometimes, we can neither prove a claim, nor disprove it!

## Task A: Picking up the shovel

**Note: You will use the data you preprocess in this task in all the subsequent ones.**

Our friends' argument concerns the entire show. We have given you a file in the `data/` folder that contains the script of every single episode. New episodes are indicated by '>>', new scenes by '>', and the rest of the lines are dialogue lines. Some lines are said by multiple people (for example, lines indicated by 'All' or 'Together'); **you must discard these lines**, for the sake of simplicity. However, you do not need to do it for Q1 in this task -- you'll take care of it when you solve Q2.

**Q1**. Your first task is to extract all lines of dialogue in each scene and episode, creating a dataframe where each row has the episode and scene where a dialogue line was said, the character who said it, and the line itself. You do not need to extract the proper name of the episode (e.g. episode 1 can appear as "Series 01 Episode 01 - Pilot Episode", and doesn't need to appear as "Pilot Episode"). Then, answer the following question: In total, how many scenes are there in each season? We're not asking about unique scenes; the same location appearing in two episodes counts as two scenes. You can use a Pandas dataframe with a season column and a scene count column as the response.

**Note: The data refers to seasons as "series".**

In [85]:
import codecs

In [86]:
# your code goes here
# load data
with codecs.open("data/all_scripts.txt" ,encoding="utf8") as f:
            all_script = f.read()

episodes = all_script.split(">>")

all_lines = []
for episode in episodes:
    lines = episode.split("\n")
    episode_lines = []
    for line in lines:
        if not line.startswith("All") and not line.startswith("Together"):
            episode_lines.append(line)
    
    all_lines.append(episode_lines)

nb_scenes = [0 for i in range(len(all_lines))]

for i, episode in enumerate(all_lines):
    for line in episode:
        if line.startswith(">"):
            nb_scenes[i]+=1

pre_df = []
for episode in all_lines:
    name = episode[0]
    for line in episode[1:]:
        pre_df.append([name, line])

pre_df = np.array(pre_df)



# create dataframe

df = pd.DataFrame(pre_df.reshape(-1,2), columns=["episode", "line"])


In [87]:
df.head()

,episode,line
0,Series 01 Episode 01 – Pilot Episode,> A corridor at a sperm bank.
1,Series 01 Episode 01 – Pilot Episode,Sheldon: So if a photon is directed through a ...
2,Series 01 Episode 01 – Pilot Episode,"Leonard: Agreed, what’s your point?"
3,Series 01 Episode 01 – Pilot Episode,"Sheldon: There’s no point, I just think it’s a..."
4,Series 01 Episode 01 – Pilot Episode,Leonard: Excuse me?


In [88]:
# J'arrive pas (( 
#df.groupby("episode").filter(lambda x: x.line.startswith(">")).count()


**Q2**. Now, let's define two sets of characters: all the characters, and recurrent characters. Recurrent characters are those who appear in more than one episode. For the subsequent sections, you will need to have a list of recurrent characters. Assume that there are no two _named characters_ (i.e. characters who have actual names and aren't referred to generically as "little girl", "grumpy grandpa", etc.) with the same name, i.e. there are no two Sheldons, etc. Generate a list of recurrent characters who have more than 90 dialogue lines in total, and then take a look at the list you have. If you've done this correctly, you should have a list of 20 names. However, one of these is clearly not a recurrent character. Manually remove that one, and print out your list of recurrent characters. To remove that character, pay attention to the _named character_ assumption we gave you earlier on. **For all the subsequent questions, you must only keep the dialogue lines said by the recurrent characters in your list.**

In [89]:
# your code goes here
df["Speaker"] = df.line.apply(lambda x: x.split(":")[0])

In [90]:
reccurent_speakers = df.groupby("Speaker").line.count().sort_values(ascending=False) >= 90
reccurent_speakers = reccurent_speakers[reccurent_speakers].index.to_list()
for wierd_speaker in ["> The apartment.", "> Penny’s apartment.", "Man", '']:
    reccurent_speakers.remove(wierd_speaker)
print(len(reccurent_speakers), reccurent_speakers)

19 ['Sheldon', 'Leonard', 'Penny', 'Howard', 'Raj', 'Amy', 'Bernadette', 'Stuart', 'Priya', 'Mrs Cooper', 'Emily', 'Beverley', 'Mrs Wolowitz', 'Zack', 'Arthur', 'Wil', 'Leslie', 'Kripke', 'Bert']


In [91]:
new_df = df[df.Speaker.isin(reccurent_speakers)]

In [92]:
new_df.head()

,episode,line,Speaker
1,Series 01 Episode 01 – Pilot Episode,Sheldon: So if a photon is directed through a ...,Sheldon
2,Series 01 Episode 01 – Pilot Episode,"Leonard: Agreed, what’s your point?",Leonard
3,Series 01 Episode 01 – Pilot Episode,"Sheldon: There’s no point, I just think it’s a...",Sheldon
4,Series 01 Episode 01 – Pilot Episode,Leonard: Excuse me?,Leonard
6,Series 01 Episode 01 – Pilot Episode,"Leonard: One across is Aegean, eight down is N...",Leonard


## Task B: Read the scripts carefully

### Part 1: Don't put the shovel down just yet

**Q3**. From each dialogue line, replace punctuation marks (listed in the EXCLUDE_CHARS variable provided in `helpers/helper_functions.py`) with whitespaces, and lowercase all the text. **Do not remove any stopwords, leave them be for all the questions in this task.**

In [99]:
# Runtime 5min 50 ...
nlp = spacy.load('en_core_web_sm')

def split_line(line):
    doc = nlp(line)
    tokens = [token.text for token in doc]
    return tokens

new_df.loc[:, "line"] = new_df.loc[:, "line"].apply(lambda line: split_line(line))
new_df.head()

/var/folders/_g/x3vmb7t16kjgc5_6vx601bh40000gn/T/ipykernel_880/1598141600.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[:, "line"] = new_df.loc[:, "line"].apply(lambda line: split_line(line))


,episode,line,Speaker
1,Series 01 Episode 01 – Pilot Episode,"[Sheldon, :, So, if, a, photon, is, directed, ...",Sheldon
2,Series 01 Episode 01 – Pilot Episode,"[Leonard, :, Agreed, ,, what, ’s, your, point, ?]",Leonard
3,Series 01 Episode 01 – Pilot Episode,"[Sheldon, :, There, ’s, no, point, ,, I, just,...",Sheldon
4,Series 01 Episode 01 – Pilot Episode,"[Leonard, :, Excuse, me, ?]",Leonard
6,Series 01 Episode 01 – Pilot Episode,"[Leonard, :, One, across, is, Aegean, ,, eight...",Leonard


**Q4**. For each term, calculate its "corpus frequency", i.e. its number of occurrences in the entire series. Visualize the distribution of corpus frequency using a histogram. Explain your observations. What are the appropriate x and y scales for this plot?

In [6]:
# your code goes here

### Part 2: Talkativity
**Q5**. For each of the recurrent characters, calculate their total number of words uttered across all episodes. Based on this, who seems to be the most talkative character?

In [7]:
# your code goes here

## Task D: The Detective's Hat

Sheldon claims that given a dialogue line, he can, with an accuracy of above 70%, say whether it's by himself or by someone else. Leonard contests this claim, since he believes that this claimed accuracy is too high.

**Q6**. Divide the set of all dialogue lines into two subsets: the training set, consisting of all the seasons except the last two, and the test set, consisting of the last two seasons.

In [8]:
# your code goes here

**Q7**. Find the set of all words in the training set that are only uttered by Sheldon. Is it possible for Sheldon to identify himself only based on these? Use the test set to assess this possibility, and explain your method.

In [9]:
# your code goes here